In [5]:
import pandas as pd
import os
import geopandas as gpd
cwd = os.getcwd()
from utils import rename_columns,filter_by_year,filter_columns,process_data


In [6]:
# configure the data
renamed_columns_list = {
    "Vaerk_ID": "plant_id",
    "År": "year",
    "Anlægstype_navn": "plant_type",
    "Værk_Kommune": "municipality_code",
    "Varmeprod_TJ": "heat_production_TJ"
}


relevant_columns =['plant_id', 'year', 'plant_type','municipality_code', 'heat_production_TJ']


In [7]:
# Read the data
df_raw = pd.read_excel('../data/EPT_DEA/EPT 1994-2022 med varmelager.xlsx', sheet_name='RawData')
df_raw= rename_columns(df_raw,renamed_columns_list)
df_raw = filter_columns(df_raw,relevant_columns)


In [8]:
# read the geojson file
gdf = gpd.read_file('../data/kommuner/kommuner.geojson')
gdf.drop(columns=['year'], inplace=True)

In [9]:
years=[2002,1994]

In [10]:

for year in years:
    df = df_raw.copy()
    df = filter_by_year(df,year)
    df = process_data(df)
    gdf_joined = gdf.merge(df, on='municipality_code', how='left')
    os.makedirs('../data/processed', exist_ok=True)
    gdf_joined.to_file(f'../data/processed/municipality_excess_heat_{year}.geojson', driver='GeoJSON')